In [22]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize,word_tokenize
from gensim.models import Word2Vec
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model
import keras.backend as K
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score

**Preparing Dataset**

In [23]:
df = pd.read_csv("Dataset/training_set_rel3.tsv", sep='\t', encoding='ISO-8859-1');
df.dropna(axis=1,inplace=True)
df.drop(columns=['domain1_score','rater1_domain1','rater2_domain1'],inplace=True,axis=1)
df.head()
temp = pd.read_csv("Processed_data.csv")
temp.drop("Unnamed: 0",inplace=True,axis=1)

In [24]:
df['domain1_score']=temp['final_score']
df.head()

,essay_id,essay_set,essay,domain1_score
0,1,1,"Dear local newspaper, I think effects computer...",6
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",7
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",5
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",8
4,5,1,"Dear @LOCATION1, I know having computers has a...",6


In [25]:
df['essay'][0]

"Dear local newspaper, I think effects computers have on people are great learning skills/affects because they give us time to chat with friends/new people, helps us learn about the globe(astronomy) and keeps us out of troble! Thing about! Dont you think so? How would you feel if your teenager is always on the phone with friends! Do you ever time to chat with your friends or buisness partner about things. Well now - there's a new way to chat the computer, theirs plenty of sites on the internet to do so: @ORGANIZATION1, @ORGANIZATION2, @CAPS1, facebook, myspace ect. Just think now while your setting up meeting with your boss on the computer, your teenager is having fun on the phone not rushing to get off cause you want to use it. How did you learn about other countrys/states outside of yours? Well I have by computer/internet, it's a new way to learn about what going on in our time! You might think your child spends a lot of time on the computer, but ask them so question about the econom

In [26]:
temp.head(1)

,essay_id,essay_set,essay,final_score,clean_essay,char_count,word_count,sent_count,avg_word_len,spell_err_count,noun_count,adj_count,verb_count,adv_count
0,1,1,"Dear local newspaper, I think effects computer...",6,Dear local newspaper I think effects computer...,1441,344,16,4.188953,11,76,75,18,24


In [27]:
#Make Dataset
y = df['domain1_score']
df.drop('domain1_score',inplace=True,axis=1)
X=df

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [29]:
X_train.shape

(9083, 3)

**PREPROCESSING**

In [30]:
train_e = X_train['essay'].tolist()
test_e = X_test['essay'].tolist()

In [31]:
train_sents=[]
test_sents=[]

stop_words = set(stopwords.words('english')) 
def sent2word(x):
    x=re.sub("[^A-Za-z]"," ",x)
    x.lower()
    filtered_sentence = [] 
    words=x.split()
    for w in words:
        if w not in stop_words: 
            filtered_sentence.append(w)
    return filtered_sentence

def essay2word(essay):
    essay = essay.strip()
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw = tokenizer.tokenize(essay)
    final_words=[]
    for i in raw:
        if(len(i)>0):
            final_words.append(sent2word(i))
    return final_words

for i in train_e:
    train_sents+=essay2word(i)

for i in test_e:
    test_sents+=essay2word(i)

In [32]:
len(train_sents)

116500

In [33]:
train_sents[0]

['It',
 'first',
 'day',
 'high',
 'school',
 'gut',
 'full',
 'butterflies',
 'make',
 'want',
 'run',
 'bathrooms',
 'hide',
 'world']

**Preparing WORD2VEC and LSTM Model**

In [34]:
def get_model():
    model = Sequential()
    model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
    model.add(LSTM(64, recurrent_dropout=0.4))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mae'])
    model.summary()
    return model

In [36]:
from gensim.models import Word2Vec

# Word2Vec parameters
num_features = 300  # Dimensionality of the word vectors
min_word_count = 40  # Minimum word count
num_workers = 4  # Number of threads to run in parallel
context = 10  # Context window size
downsampling = 1e-3  # Downsample setting for frequent words

# Train Word2Vec model
model = Word2Vec(sentences=train_sents, 
                 workers=num_workers, 
                 vector_size=num_features,  # Updated from 'size' to 'vector_size'
                 min_count=min_word_count, 
                 window=context, 
                 sample=downsampling)

# Save the model in binary format
model.wv.save_word2vec_format('word2vecmodel.bin', binary=True)


In [38]:
import numpy as np

# Function to average the word vectors for a sentence
def makeVec(words, model, num_features):
    vec = np.zeros((num_features,), dtype="float32")
    noOfWords = 0
    # Using 'key_to_index' instead of 'index2word'
    index2word_set = set(model.wv.key_to_index)
    
    for i in words:
        if i in index2word_set:
            noOfWords += 1
            vec = np.add(vec, model.wv[i])  # Access word vectors via 'model.wv'
    
    # To avoid division by zero in case no words match
    if noOfWords > 0:
        vec = np.divide(vec, noOfWords)
    return vec

# Function to transform each essay into its average word vector
def getVecs(essays, model, num_features):
    essay_vecs = np.zeros((len(essays), num_features), dtype="float32")
    for idx, essay in enumerate(essays):
        essay_vecs[idx] = makeVec(essay, model, num_features)
    return essay_vecs

# Assuming 'train_e' and 'test_e' are lists of essays and 'sent2word' tokenizes the essays
clean_train = [sent2word(i) for i in train_e]
clean_test = [sent2word(i) for i in test_e]

# Getting vectors for train and test data
training_vectors = getVecs(clean_train, model, num_features)
testing_vectors = getVecs(clean_test, model, num_features)


In [39]:
training_vectors.shape

(9083, 300)

In [40]:
training_vectors = np.array(training_vectors)
testing_vectors = np.array(testing_vectors)

# Reshaping train and test vectors to 3 dimensions. (1 represnts one timestep)
training_vectors = np.reshape(training_vectors, (training_vectors.shape[0], 1, training_vectors.shape[1]))
testing_vectors = np.reshape(testing_vectors, (testing_vectors.shape[0], 1, testing_vectors.shape[1]))
lstm_model = get_model()

c:\Users\visio\OneDrive\Desktop\essay\essay\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 1, 300)         │       721,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)             │        93,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 814,705 (3.11 MB)

 Trainable params: 814,705 (3.11 MB)

 Non-trainable params: 0 (0.00 B)

In [41]:
training_vectors.shape

(9083, 1, 300)


**TRAINING AND PREDICTION**







In [42]:
lstm_model.fit(training_vectors, y_train, batch_size=64, epochs=150)

Epoch 1/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 12.1608 - mae: 2.7504
Epoch 2/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.4664 - mae: 1.8493
Epoch 3/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.0946 - mae: 1.7774
Epoch 4/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.8855 - mae: 1.7307
Epoch 5/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.5851 - mae: 1.6842
Epoch 6/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.6118 - mae: 1.6863
Epoch 7/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.5290 - mae: 1.6752
Epoch 8/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 4.4122 - mae: 1.6410
Epoch 9/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.3629 - mae: 1.6456
Epoch 10/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.2522 - mae: 1.6136
Epoch 11/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.0674 - mae: 1.5793
Epoch 12/150
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 4.0414 - mae: 1.5728
Epoch 13/150

In [43]:
lstm_model.save('final_lstm.h5')
y_pred = lstm_model.predict(testing_vectors)
y_pred = np.around(y_pred)
y_pred

122/122 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step


array([[3.],
       [5.],
       [7.],
       ...,
       [7.],
       [8.],
       [9.]], dtype=float32)